<a href="https://colab.research.google.com/github/PosgradoMNA/actividades-del-projecto-equipo_36/blob/main/Reto_Aguas_subterraneas_Entrega2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Ciencia y Analítica de Datos**
###Tecnológico de Monterrey
###Prof Titular: **Dra. María de la Paz Rico**
###Tutor: **Roberto Antonio Guevara González**

####**Equipo 36**
####*Marcos Chávez - A01688507*
####*Bernanrdo Mijangos - A01793654*

https://github.com/PosgradoMNA/actividades-del-projecto-equipo_36/blob/main/Reto_Aguas_subterraneas_Entrega1.ipynb

####*Fecha de entrega 18/11/2022*

###**Base de datos Aguas Subterraneas**

In [26]:
from sklearn import linear_model as lm
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as sts
import pandas as pd
import scipy as scp
import numpy as np
import sklearn.preprocessing
from sklearn.model_selection import train_test_split  ### for train and test split package
from sklearn import metrics  ## For calculation of MSE & RMSE
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, f1_score

import requests, zipfile
from io import BytesIO

from sklearn.cluster import KMeans
from tqdm import tqdm
from shapely.geometry import Point
from yellowbrick.cluster import SilhouetteVisualizer, KElbowVisualizer
from sklearn.metrics import silhouette_score
from geopy.geocoders import Nominatim
import geopy.distance

from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.colors

In [2]:
url = 'http://201.116.60.46/Datos_de_calidad_del_agua_de_5000_sitios_de_monitoreo.zip'

In [3]:
req = requests.get(url)
zipfile.ZipFile(BytesIO(req.content)).extractall('unzipped_zip/')
df=pd.read_csv('unzipped_zip/Datos_de_calidad_del_agua_2020/Datos_de_calidad_del_agua_de_sitios_de_monitoreo_de_aguas_subterraneas_2020.csv', encoding = 'latin1')
df.head()

,CLAVE,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,ACUIFERO,SUBTIPO,LONGITUD,LATITUD,PERIODO,...,CUMPLE_CON_DUR,CUMPLE_CON_CF,CUMPLE_CON_NO3,CUMPLE_CON_AS,CUMPLE_CON_CD,CUMPLE_CON_CR,CUMPLE_CON_HG,CUMPLE_CON_PB,CUMPLE_CON_MN,CUMPLE_CON_FE
0,DLAGU6,POZO SAN GIL,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,ASIENTOS,VALLE DE CHICALOTE,POZO,-102.02210,22.20887,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
1,DLAGU6516,POZO R013 CAÑADA HONDA,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,AGUASCALIENTES,VALLE DE CHICALOTE,POZO,-102.20075,21.99958,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
2,DLAGU7,POZO COSIO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,COSIO,VALLE DE AGUASCALIENTES,POZO,-102.28801,22.36685,2020,...,SI,SI,SI,NO,SI,SI,SI,SI,SI,SI
3,DLAGU9,POZO EL SALITRILLO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,RINCON DE ROMOS,VALLE DE AGUASCALIENTES,POZO,-102.29449,22.18435,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
4,DLBAJ107,RANCHO EL TECOLOTE,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LA PAZ,TODOS SANTOS,POZO,-110.24480,23.45138,2020,...,SI,SI,NO,SI,SI,SI,SI,SI,SI,SI


In [5]:
binarias = ['CUMPLE_CON_ALC', 'CUMPLE_CON_COND', 'CUMPLE_CON_SDT_ra',
            'CUMPLE_CON_SDT_salin', 'CUMPLE_CON_FLUO', 'CUMPLE_CON_DUR',
            'CUMPLE_CON_CF', 'CUMPLE_CON_NO3', 'CUMPLE_CON_AS', 'CUMPLE_CON_CD',
            'CUMPLE_CON_CR', 'CUMPLE_CON_HG', 'CUMPLE_CON_PB', 'CUMPLE_CON_MN',
            'CUMPLE_CON_FE']

###**Selecciona tus variables independientes X y dependiente Y (semáforo)**

In [23]:
X = df[binarias]
Y = df['SEMAFORO']

In [28]:
import sklearn.preprocessing as preprocessing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [15]:
le = preproc.LabelEncoder()

In [22]:
df[calidad].describe()

,SEMAFORO
count,1068
unique,3
top,Verde
freq,434


In [8]:
onehot_enc = Pipeline(steps=[('onehot_encoder', OneHotEncoder())])
label_enc = Pipeline(steps=[('label_encoder', LabelEncoder())])


In [9]:
col_transform = ColumnTransformer(transformers= [('onehot_encoder', onehot_enc, calidad),
                                                 ('label_encoder', label_enc, binarias)])

In [53]:
#X_temp = col_transform.fit_transform(df)

###**Cambia a label encoding el semáforo, ej, de ["clase 1", "clase 2", "clase 3"] a [ 1,2,3]**

In [29]:
le = preprocessing.LabelEncoder()

In [38]:
le.fit(['Verde', 'Amarillo', 'Rojo'])

LabelEncoder()

In [39]:
list(le.classes_)

['Amarillo', 'Rojo', 'Verde']

In [40]:
le.transform(Y)

array([2, 2, 1, ..., 1, 2, 2])

In [50]:
 Y_le = pd.DataFrame(le.transform(Y))
 Y_le.value_counts()

2    434
1    387
0    247
dtype: int64

In [52]:
Y.value_counts()

Verde       434
Rojo        387
Amarillo    247
Name: SEMAFORO, dtype: int64

In [ ]:
le = preprocessing.LabelEncoder()


###**Realiza un análisis general de las features importances a traves de decision trees o random forest**

###**Selecciona las variables de mayor importancia**

###**Realiza tu clasificador, recuerda dividir los datos de manera balanceada (auxiliate de train test split)**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

###**Explora que clasificador es el más optimo, ejemplo:**
###-*Decision trees*
###-*Random Forest*

###**Determina el grado de exactitud a través del reporte de clasificación https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html**

###**Visualiza los resultados del modelo o las predicciones a través de una matriz de confusión**